In [37]:
import pandas as pd
import json

In [38]:
df = pd.read_csv('aiddata-countries-only.csv')
df.head()

,Unnamed: 0,aiddata_id,aiddata_2_id,year,donor,recipient,commitment_amount_usd_constant,coalesced_purpose_code,coalesced_purpose_name
0,1,2414478.0,NaN,1977,Saudi Arabia,India,348718518.0,23030,Power generation/renewable sources
1,2,2414509.0,NaN,1977,Saudi Arabia,Brazil,191647004.0,23040,Electrical transmission/ distribution
2,3,2414635.0,NaN,1983,Saudi Arabia,India,79371799.0,21030,Rail transport
3,4,2414665.0,NaN,1984,Saudi Arabia,Taiwan,212202942.0,21030,Rail transport
4,5,2414667.0,NaN,1984,Saudi Arabia,Korea,134511154.0,21040,Water transport


In [39]:
# I have decided to drop sectors (for now...) because the categoy labels are too low-level and there are >80000 unspecified sectors
df['coalesced_purpose_name'].value_counts()

Sectors not specified                                                                          8076
Social/ welfare services                                                                       6228
Strengthening civil society                                                                    5600
Higher education                                                                               4593
Multisector aid                                                                                3026
Material relief assistance and services                                                        2243
Education facilities and training                                                              2144
Administrative costs                                                                           1866
Agricultural development                                                                       1654
Vocational training                                                                            1613


In [40]:
# Sanity check: Each row represents a unique year, donor, recipient and sector
df[(df['donor'] == 'United States') &(df['year'] == 1977)]

,Unnamed: 0,aiddata_id,aiddata_2_id,year,donor,recipient,commitment_amount_usd_constant,coalesced_purpose_code,coalesced_purpose_name
62322,62323,84788704.0,6613119.0,1977,United States,Korea,124945894.0,52010,Food security programmes/food aid
62732,62733,84985067.0,6520467.0,1977,United States,Thailand,720668.0,60010,Action relating to debt
63336,63337,85304884.0,6529786.0,1977,United States,Korea,6125682.0,60010,Action relating to debt
63958,63959,85627686.0,6519458.0,1977,United States,Thailand,9008356.0,31181,Agricultural education/training
64894,64895,86129314.0,6539424.0,1977,United States,Thailand,13512534.0,43040,Rural development
64922,64923,86142340.0,6529342.0,1977,United States,Korea,5855431.0,60010,Action relating to debt
65288,65289,86338964.0,6529675.0,1977,United States,Chile,180167.0,60010,Action relating to debt
65776,65777,86632775.0,6520060.0,1977,United States,Korea,1441337.0,60010,Action relating to debt
66560,66561,87035484.0,6520430.0,1977,United States,Korea,9879163.0,60010,Action relating to debt
67222,67223,87377778.0,6520134.0,1977,United States,Korea,360334.0,60010,Action relating to debt


In [41]:
#Group by year, donor, recipient and sum up the commitment amount
df = df[['year', 'donor', 'recipient', 'commitment_amount_usd_constant']]
df_new = df.groupby(['year', 'donor', 'recipient']).sum().reset_index()
df_new = df_new.reset_index().rename(columns={'index':'id', 'commitment_amount_usd_constant': 'amount'})

#Convert year from int to string
df_new['year'] = df_new['year'].astype(str)
df_new.head()

,id,year,donor,recipient,amount
0,0,1973,Australia,India,36565064.0
1,1,1973,Australia,Korea,1162012.0
2,2,1973,Australia,Thailand,8558787.0
3,3,1973,Belgium,Colombia,6541889.0
4,4,1973,Belgium,India,32709447.0


In [42]:
df_new.year.unique()

array(['1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980',
       '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988',
       '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013'], dtype=object)

In [43]:
def flatten_multi_index(df):
    mi = df.columns
    suffixes, prefixes = mi.levels
    col_names = [prefixes[i_p] + '_' + suffixes[i_s] for (i_s, i_p) in zip(*mi.labels)]
    df.columns = col_names
    return df

In [44]:
df_new.recipient.unique()

array(['India', 'Korea', 'Thailand', 'Colombia', 'Brazil', 'Chile',
       'Cyprus', 'Kuwait', 'Saudi Arabia', 'United Arab Emirates',
       'Taiwan', 'Australia', 'Austria', 'Belgium', 'Canada', 'Denmark',
       'France', 'Greece', 'Iceland', 'Ireland', 'Italy', 'Japan',
       'New Zealand', 'Norway', 'Poland', 'Portugal', 'Romania', 'Spain',
       'Sweden', 'United Kingdom', 'Liechtenstein', 'Finland', 'Hungary',
       'South Africa', 'Qatar', 'Czech Republic', 'Latvia', 'Estonia',
       'Lithuania', 'Slovenia', 'Slovak Republic', 'Luxembourg',
       'Switzerland', 'United States', 'Germany'], dtype=object)

## Data Processing - Global Distribution of Donors/Recipients

In [209]:
class DataProcessing_GlobalDistribution():
    
    def __init__(self):
        
        """
        Entities could be: donor/recipient
        This class contains functions to calculate the total sum of commitment amount as either a donor/recipient on a yearly basis
        """

    def final(self, df):

        # [DESTINATION]
        destination_count = self.global_stats(df, 'recipient')

        # [SOURCE]
        source_count = self.global_stats(df, 'donor')

        # Global count
        all_country = pd.concat([source_count, destination_count])
        all_country_PT = self.country_count_PT(all_country)
      
        for c in ['donor', 'recipient']:
            for country in list(set(list(df.donor.unique()) + list(df.recipient.unique()))):
                for i in range(1973, 2014):
                    tmp = all_country[(all_country['year'] == i) & (all_country['country'] == country) & (all_country['category'] == c)]
                    if(len(tmp)==0):
                        new = pd.DataFrame({'year': i, 'country': country, 'category': c, 'sum':0}, index=[0])
                        all_country = pd.concat([all_country, new])
    
        all_country_all = pd.DataFrame(all_country.groupby(['category','year'])['sum'].sum()).reset_index()
        all_country_all['country'] = 'All'
        all_country = pd.concat([all_country, all_country_all])
        all_country = all_country.sort_values(['category', 'year'], ascending=True).reset_index(drop=True)
        
        # Sum
        net_all = pd.DataFrame()
        for country in list(set(list(df_new.donor.unique()) + list(df_new.recipient.unique()))):
            tmp = all_country_PT[all_country_PT['country'] == country]
            donor = tmp[tmp['category'] == 'donor'].drop(['country', 'category'], axis=1).reset_index(drop=True)
            recipient = tmp[tmp['category'] == 'recipient'].drop(['country', 'category'], axis=1).reset_index(drop=True)
            if((len(donor)>0) & (len(recipient)>0)):
                net = donor.add(recipient)
            elif (len(donor)==0):
                net = recipient 
            elif (len(recipient)==0):
                net = donor 
            net['country'] = country
            net_all = pd.concat([net_all, net])

        net_all['category'] = 'net'
        net_all = net_all.fillna(0).reset_index(drop=True)
        all_country_PT = pd.concat([all_country_PT, net_all])

        # Global density (in %)
        all_country_pctPT = self.country_pct_PT(all_country)

        return all_country, all_country_PT, all_country_pctPT
    
    def global_stats(self, df, category):

        country_stats = pd.DataFrame()

        tmp_g1 = pd.DataFrame(df.groupby([category])['amount'].sum()).reset_index().rename(columns={'amount': 'sum'}) 
        tmp_g1['year'] = "All"
        tmp_pcts = pd.DataFrame(df.groupby([category])['amount'].sum()).apply(lambda x: 100 * x / float(x.sum())).reset_index().rename(columns={'amount': 'pct'})
        tmp_g1['year'] = "All"
        new = pd.merge(tmp_g1, tmp_pcts, on=category, how='left')
        country_stats = pd.concat([country_stats, new])
        
        # segment by year
        for i in df['year'].unique():
            tmp = df[df['year'] == i]
            if(len(tmp) > 0):
                tmp_g1 = pd.DataFrame(tmp.groupby([category])['amount'].sum()).reset_index().rename(columns={'amount': 'sum'})     
                tmp_g1['year'] = int(i)
                tmp_pcts =  pd.DataFrame(tmp.groupby([category])['amount'].sum()).apply(lambda x: 100 * x / float(x.sum())).reset_index().rename(columns={'amount': 'pct'})
                tmp_pcts['year'] = int(i)
                tmp_pcts = pd.merge(tmp_g1, tmp_pcts, on=[category, 'year'], how='left')
                country_stats = pd.concat([country_stats, tmp_pcts])
                
        country_stats['category'] = category
        country_stats = country_stats.rename(columns={category: 'country'})
        return country_stats

    def country_count_PT(self, df):
        
        all_stats_PT = pd.pivot_table(df, values='sum', index=['country', 'category'], columns=['year']) 
        all_stats_PT.fillna(0, inplace=True)
        return all_stats_PT.reset_index()

    def country_pct_PT(self, df):

        all_stats_PT = pd.pivot_table(df, values='pct', index=['country', 'category'], columns=['year']) 
        all_stats_PT.fillna(0, inplace=True)
        return all_stats_PT.reset_index()
    
    
    def rollup_time_count(self, df, groupby_list):

        df_time = pd.DataFrame(df.groupby(groupby_list)['student_since'].count()).reset_index()
        df_time['rollup'] = df_time.loc[0, 'student_since']
        for i,row in df_time.iterrows():
            try:
                df_time.loc[i+1, 'rollup'] = df_time.loc[i, 'rollup'] + df_time.loc[i+1, 'student_since'] 
            except:
                pass

        df_time['time'] = df_time.apply(lambda row: str(row['year']) + '-' + str(row['semester']), axis=1)
        return df_time
    

In [210]:
gd = DataProcessing_GlobalDistribution()

In [211]:
all_country, all_country_PT, all_country_pctPT = gd.final(df_new)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [79]:
all_country_pctPT[["All"]].values.sum()

200.00000000000003

In [213]:
all_country.to_csv('all_country.csv', index=False)
all_country_PT.to_csv('all_country_PT.csv', index=False)
all_country_pctPT.to_csv('all_country_pctPT.csv', index=False)

In [204]:
all_country[(all_country['country'] == 'Poland') & (all_country['category'] == 'recipient')]

,category,country,pct,sum,year
2051,recipient,Poland,NaN,0.000000e+00,1973
2098,recipient,Poland,NaN,0.000000e+00,1974
2146,recipient,Poland,NaN,0.000000e+00,1975
2194,recipient,Poland,NaN,0.000000e+00,1976
2242,recipient,Poland,NaN,0.000000e+00,1977
2291,recipient,Poland,NaN,0.000000e+00,1978
2339,recipient,Poland,NaN,0.000000e+00,1979
2370,recipient,Poland,0.676608,7.500045e+07,1980
2417,recipient,Poland,0.000254,3.541000e+04,1981
2459,recipient,Poland,0.000890,7.032600e+04,1982


In [212]:
all_country[(all_country['country'] == 'All')]

,category,country,pct,sum,year
48,donor,All,NaN,4.815472e+09,1973
97,donor,All,NaN,6.805438e+09,1974
146,donor,All,NaN,4.251300e+09,1975
195,donor,All,NaN,3.695818e+09,1976
244,donor,All,NaN,4.928006e+09,1977
293,donor,All,NaN,4.798917e+09,1978
342,donor,All,NaN,3.313050e+09,1979
391,donor,All,NaN,1.108477e+10,1980
440,donor,All,NaN,1.391494e+10,1981
489,donor,All,NaN,7.898765e+09,1982


## Data Processing - Country-To-Country Connections

In [80]:
class DataProcessing_CountryToCountry():
    
    def __init__(self):
        
        """
        A country could be:
        1) Donor
        2) Recipient
        3) Both risk source and destination
        This is a class containing functions to find the density of connections a country has to other countries based on relationship on a yearly basis 
        """

    def final(self, df):
     
        """
        Output: Transformed data ready for input into chart
        
        Parameters:
        df: Cleaned dataframe of each relationship between country
        """
      
        wn_country_donor = self.country_to_country(df, ['donor','recipient'])                                                                      
        wn_country_recipient = self.country_to_country(df, ['recipient', 'donor'])   
        wn_country_net = self.net(df)
        wn_country_PT = self.country_flow_PT(wn_country_donor, wn_country_recipient, wn_country_net)
        return wn_country_PT
    
    def country_to_country(self, df, X):

        """
        Output: Count and percentage of connections to each country, segmented by year
        """
        
        country_stats = pd.DataFrame()

        tmp = pd.DataFrame(df.groupby(X)['amount'].sum()).reset_index()
        tmp['year'] = 'All'
        country_stats = pd.concat([country_stats, tmp])
        
        # segment by year
        for i in df['year'].unique():
            tmp = df[df['year'] == i]
            if(len(tmp) > 0):
                tmp = pd.DataFrame(tmp.groupby(X)['amount'].sum()).reset_index()    
                tmp['year'] = str(int(i))
                country_stats = pd.concat([country_stats, tmp])

        country_stats = country_stats[['year', 'donor', 'recipient', 'amount']]
        
        return country_stats

    def net(self, df):
        
        rsAll1= pd.DataFrame(df.groupby(['donor','recipient'])['amount'].sum()).reset_index()
        rsAll1 = rsAll1.rename(columns={'recipient': 'country 2', 'donor': 'country 1', 'amount': 'outflow'})
        rsAll1['year'] = 'All'
        rsAll2 = pd.DataFrame(df.groupby(['recipient', 'donor'])['amount'].sum()).reset_index()
        rsAll2 = rsAll2.rename(columns={'donor': 'country 2', 'recipient': 'country 1', 'amount': 'inflow'})
        rsAll2['year'] = 'All'
        
        rs1 = pd.DataFrame()
        rs2 = pd.DataFrame()
        for i in list(set(list(df.donor.unique()) + list(df.recipient.unique()))):
            tmp1 = df[(df['donor'] == i)]
            rs1 = pd.concat([rs1, tmp1])
            tmp2 = df[(df['recipient'] == i)]
            rs2 = pd.concat([rs2, tmp2])

        rs1 = rs1.rename(columns={'recipient': 'country 2', 'donor': 'country 1', 'amount': 'outflow'})
        rs2 = rs2.rename(columns={'donor': 'country 2', 'recipient': 'country 1', 'amount': 'inflow'})
    
        net = pd.merge(rs1, rs2, on=['country 1', 'country 2', 'year'], how='left')
        net = pd.concat([net, rsAll1, rsAll2])
        net['inflow'] = net['inflow'].fillna(0)
        net['outflow'] = net['outflow'].fillna(0)
        net['net'] = net['outflow'] - net['inflow']
        net['value'] = net['outflow'] + net['inflow']
        net = net.rename(columns={'country 1': 'country'})
        
        return net[['country', 'country 2', 'inflow', 'outflow', 'year', 'net', 'value']]
    
    def country_flow_PT(self, df1, df2, df3):

        df1_PT = pd.pivot_table(df1, values='amount', index=['donor'], columns=['year', 'recipient']) 
        df1_PT = flatten_multi_index(df1_PT).reset_index().rename(columns={'donor': 'country'})
        df1_PT['category'] = 'recipient'

        df2_PT = pd.pivot_table(df2, values='amount', index=['recipient'], columns=['year', 'donor']) 
        df2_PT = flatten_multi_index(df2_PT).reset_index().rename(columns={'recipient': 'country'})
        df2_PT['category'] = 'donor'

        df3_PT = pd.pivot_table(df3, values='net', index=['country'], columns=['year', 'country 2']) 
        df3_PT = flatten_multi_index(df3_PT).reset_index()
        df3_PT['category'] = 'net'
        
        df = pd.concat([df1_PT, df2_PT, df3_PT])
        df_PT = pd.pivot_table(df, index=['country'], columns=['category']) 
        df_PT = flatten_multi_index(df_PT).reset_index()
        df_PT = df_PT.fillna(0)

        return df_PT

In [81]:
ctc = DataProcessing_CountryToCountry()

In [82]:
wn_country_PT = ctc.final(df_new)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:73: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:96: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [83]:
wn_country_PT.head()

,country,donor_Australia_1973,donor_Australia_1974,donor_Australia_1975,donor_Australia_1976,donor_Australia_1977,donor_Australia_1978,donor_Australia_1979,donor_Australia_1980,net_Australia_1980,...,net_United States_2008,recipient_United States_2008,donor_United States_2009,donor_United States_2010,donor_United States_2011,donor_United States_2012,donor_United States_2013,donor_United States_All,net_United States_All,recipient_United States_All
0,Australia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.572618e+09,-2.572618e+09,0.0
1,Austria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.699085e+08,-7.699085e+08,0.0
2,Belgium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.843462e+08,-1.843462e+08,0.0
3,Brazil,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,55528517.0,31086679.0,33062881.0,21104514.0,49846811.0,1.354834e+10,-1.354834e+10,0.0
4,Canada,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.068597e+09,-4.068597e+09,0.0


In [42]:
max(wn_country_PT.drop(['country'], axis=1).max(axis = 0).values)

48830067295.0

In [18]:
wn_country_PT.to_csv('wn_country_PT.csv', index=False) 

In [78]:
class DataProcessing_CountrySnapshot():
    
    def __init__(self):
        
        """
        Country-level metrics
        """

    def countries_ranking(self, df_filt):
     
        """
        To rank countries connected to each country either as a donor/recipient based on commitment amount
        
        Parameters:
        df: Cleaned dataframe of each relationship between country
        """
    
        #[SOURCE]
        top_exposing = df_filt.groupby(['recipient', 'donor'])['amount'].sum().reset_index()
        top_exposing = top_exposing.sort_values('amount',ascending=False)
        top_exposing['category'] = 'recipient'
        
        #[DESTINATION]
        top_exposedby = df_filt.groupby(['donor', 'recipient'])['amount'].sum().reset_index()
        top_exposedby = top_exposedby.sort_values('amount',ascending=False)
        top_exposedby['category'] = 'donor'

        countries_ranking = pd.concat([top_exposing, top_exposedby])
        countries_ranking = countries_ranking.rename(columns={'amount': 'sum'})

        return countries_ranking
   

In [79]:
country = DataProcessing_CountrySnapshot()
countries_ranking = country.countries_ranking(df_new)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [80]:
countries_ranking.to_csv('countries_ranking.csv', index=False)

In [81]:
countries_ranking

,sum,category,donor,recipient
197,4.883007e+10,recipient,Japan,India
466,3.543769e+10,recipient,Japan,Thailand
215,1.992198e+10,recipient,United Kingdom,India
192,1.801798e+10,recipient,Germany,India
301,1.723075e+10,recipient,Germany,Poland
94,1.710424e+10,recipient,United States,Colombia
216,1.652983e+10,recipient,United States,India
32,1.354834e+10,recipient,United States,Brazil
231,1.302945e+10,recipient,Japan,Korea
238,1.115436e+10,recipient,United States,Korea


In [6]:
asia = json.load(open('asia.geo.json', 'r'))        

In [16]:
for i in asia['features']:
    print(i['properties']['name'])

China
Azerbaijan
Afghanistan
Bhutan
Brunei
Bangladesh
Armenia
United Arab Emirates
N. Cyprus
Cyprus
Georgia
Indonesia
India
Iran
Israel
Iraq
Jordan
Japan
Kazakhstan
Kyrgyzstan
Cambodia
Korea
Kuwait
Lao PDR
Lebanon
Sri Lanka
Myanmar
Mongolia
Malaysia
Nepal
Oman
Pakistan
Philippines
Palestine
Dem. Rep. Korea
Qatar
Saudi Arabia
Syria
Tajikistan
Uzbekistan
Timor-Leste
Turkey
Taiwan
Thailand
Turkmenistan
Vietnam
Yemen
Australia
Papua New Guinea
New Caledonia
Vanuatu
Fiji
Solomon Is.
New Zealand


In [67]:
world_economies = pd.read_csv('world_bank_economies.csv')
world_economies['Economy'] = world_economies['Economy'].replace('Hong Kong SAR, China', 'Hong Kong')
world_economies['Economy'] = world_economies['Economy'].replace('Macao SAR, China', 'Macau')
world_economies['Economy'] = world_economies['Economy'].replace('Lao PDR', 'Laos')
world_economies['Economy'] = world_economies['Economy'].replace('Taiwan, China', 'Taiwan')
countries = world_economies[world_economies['Region'].isin(['South Asia', 'East Asia & Pacific'])]['Economy'].unique()

In [70]:
countries

array(['Afghanistan', 'American Samoa', 'Australia', 'Bangladesh',
       'Bhutan', 'Brunei Darussalam', 'Cambodia', 'China', 'Fiji',
       'French Polynesia', 'Guam', 'Hong Kong', 'India', 'Indonesia',
       'Japan', 'Kiribati', "Korea, Dem. People's Rep.", 'Korea, Rep.',
       'Laos', 'Macau', 'Malaysia', 'Maldives', 'Marshall Islands',
       'Micronesia, Fed. Sts.', 'Mongolia', 'Myanmar', 'Nauru', 'Nepal',
       'New Caledonia', 'New Zealand', 'Northern Mariana Islands',
       'Pakistan', 'Palau', 'Papua New Guinea', 'Philippines', 'Samoa',
       'Singapore', 'Solomon Islands', 'Sri Lanka', 'Taiwan', 'Thailand',
       'Timor-Leste', 'Tonga', 'Tuvalu', 'Vanuatu', 'Vietnam'],
      dtype=object)

In [48]:
df_new[df_new['recipient'].isin(countries)].recipient.unique()

array(['India', 'Thailand', 'Australia', 'Japan', 'New Zealand'],
      dtype=object)

In [51]:
df_new[df_new['donor'].isin(countries)].donor.unique()

array(['Australia', 'Japan', 'New Zealand', 'Thailand'], dtype=object)